In [4]:
import re
from nltk.tokenize import sent_tokenize

side_effects_sub_section_xml_pattern = re.compile('===Side effects===|===Adverse effects===')
side_effects_main_section_xml_pattern = re.compile('==Side effects==|==Adverse effects==')
side_effects_random_section_xml_pattern = re.compile('^&lt;!-- (.*effects.*) --&gt;')
special_tags_1 = re.compile('&lt;ref&gt;((.|\n)*?)&lt;\/ref&gt;')
special_tags_2 = re.compile('&lt;ref name((.|\n)*?)\/&gt;')
special_tags_3 = re.compile('β&lt;((.)*?)\/sub&gt;')
special_tags_4 = re.compile('&lt;ref((.|\n)*?)&lt;\/ref&gt;')
effects_pattern = re.compile('(((\S+\W){2,15})((e|E)ffects?)|((H|h)hazards?)|((c|C)auses?)((\W\S+){2,5}))')
isSubSection = False
sub_section_xml_pattern = re.compile('[^=]===[^=]')
main_section_xml_pattern = re.compile('[^=]==[^=]')
extraction = False
article_title = ''
extracted_text = ''
no_effect_sections = ''
number_of_sections_extracted = 0
total_drugs = 0
total_pulled = 0
total_article_lines = 0
smaller_articles_count = 0

output_file = open('C:/Users/kavya/Desktop/mds/pubmed/first_output.xml', 'w',encoding="utf8")

wiki_file = open('C:/Users/kavya/Desktop/mds/pubmed/Related_articles.xml', 'r',encoding="utf8")
total_lines_count = sum(1 for _ in wiki_file)

wiki_file = open('C:/Users/kavya/Desktop/mds/pubmed/Related_articles.xml', 'r',encoding="utf8")

for line_count, line in enumerate(wiki_file):
    result = re.search('<title>',line)
    if result:
        total_drugs += 1
        article_title = line
        if extraction:
            extracted_text = re.sub(special_tags_1,'', extracted_text)
            extracted_text = re.sub(special_tags_2,'', extracted_text)
            extracted_text = re.sub(special_tags_3,'', extracted_text)
            extracted_text = re.sub(special_tags_4,'', extracted_text)
#             print(extracted_text)
            output_file.write(extracted_text)
            total_pulled += 1
            extracted_text = ''
            extraction = False
            isSubSection = False
            number_of_sections_extracted += 1
        else:
            if no_effect_sections:
                no_effect_sections = article_title + '\n' + no_effect_sections
                no_effect_sections = re.sub(special_tags_1,'', no_effect_sections)
                no_effect_sections = re.sub(special_tags_2,'', no_effect_sections)
                no_effect_sections = re.sub(special_tags_3,'', no_effect_sections)
                no_effect_sections = re.sub(special_tags_4,'', no_effect_sections)
                output_file.write(no_effect_sections)
                total_pulled += 1
                no_effect_sections = ''
            else:
                if total_article_lines < 100:
                    smaller_articles_count += 1
                total_article_lines = 0
    if not extraction and side_effects_main_section_xml_pattern.search(line):
        isSubSection = False
        extraction = True
        extracted_text += article_title + '\n'
        continue
    elif not extraction and side_effects_sub_section_xml_pattern.search(line):
        isSubSection = True
        extraction = True
        extracted_text += article_title + '\n'
        continue
    if extraction:
        no_effect_sections = ''
        total_article_lines = 0
#         print('extraction section: '+line)
        if main_section_xml_pattern.search(line) or sub_section_xml_pattern.search(line):
            if isSubSection:
                extracted_text = re.sub(special_tags_1,'', extracted_text)
                extracted_text = re.sub(special_tags_2,'', extracted_text)
                extracted_text = re.sub(special_tags_3,'', extracted_text)
                extracted_text = re.sub(special_tags_4,'', extracted_text)
#                 print(extracted_text)
                output_file.write(extracted_text)
                total_pulled += 1
                extracted_text = ''
                extraction = False
                isSubSection = False
                number_of_sections_extracted += 1
            elif main_section_xml_pattern.search(line):
                extracted_text = re.sub(special_tags_1,'', extracted_text)
                extracted_text = re.sub(special_tags_2,'', extracted_text)
                extracted_text = re.sub(special_tags_3,'', extracted_text)
                extracted_text = re.sub(special_tags_4,'', extracted_text)
#                 print(extracted_text)
                output_file.write(extracted_text)
                total_pulled += 1
                extracted_text = ''
                extraction = False
                number_of_sections_extracted += 1
            else:
                extracted_text += line
        else:
            extracted_text += line
    else:
#         print(line)
        total_article_lines += len(line)
        
        searc = re.search(r'(( (E|e)ffects? )|( (C|c)auses? )|((H|h)hazards?)|( decreases?d? )|( increases?d? ))', line)
    
        if searc:
#             print('Length of line is :' + str(len(line)))
            
            start_index = 0 if searc.start() < 250 else searc.start() - 250
            
            end_index = len(line) if (searc.start() + 250) > len(line) else searc.start() + 250
#             print('End of line is :' + str(end_index))
#             print(searc.start())
            counter = searc.start()
            
            temp = ''
            while counter > start_index:
                temp = temp + line[counter]
                counter-=1
                
            temp = temp + '\n'
            counter = searc.start()
            while counter < end_index:
                temp = temp + line[counter]
                counter+=1
                
            temp = temp + '\n'
            
#             print(temp)
            no_effect_sections += temp
            

print("Number of articles present are: " + str(total_drugs))
print("Number of sections extracted are: " + str(total_pulled))
print("Number of smaller articles are: " + str(total_pulled))

## Commented trials
#         no_effect_sections_temp = sent_tokenize(line)
#         no_effect_sections += '--::@@::--'.join(no_effect_sections_temp)
            
# print("Number of sections extracted are: " + str(number_of_sections_extracted))

#             print(no_effect_sections)
#             all_lines = no_effect_sections.split('--::@@::--')
#             line_count = len(all_lines)
#             count = 0
#             temp = ''
#             print('total line count is :'+str(line_count))
            
#             while count < line_count:
        
#                 for match in re.findall(r'(((\S+\W){2,15})((e|E)ffects?|(c|C)auses?)((\W\S+){2,15}))', all_lines[count - 1]):
#                     if match:
#                         temp = temp + match + '\n'
                    
#                 print('current line count is :'+str(count))
#                 if 'effects ' in all_lines[count - 1] or 'effect ' in all_lines[count - 1] or ' causes ' in all_lines[count - 1] or ' cause ' in all_lines[count - 1]:
#                     if count + 5 < line_count:
#                         temp = '\n'.join(all_lines[count - 1:])
#                         count = line_count
#                     else:
#                         temp = '\n'.join(all_lines[count - 1:count + 4])
#                         count = count + 5
#                 else:
#                     count+=1
            
#             print(temp)
#             if temp != '':
#                 temp = re.sub(special_tags_1,'', temp)
#                 temp = re.sub(special_tags_2,'', temp)
#                 temp = re.sub(special_tags_3,'', temp)
#                 temp = re.sub(special_tags_4,'', temp)
#                 temp = article_title + '\n' + temp + '\n'
#                 output_file.write(temp)

Number of articles present are: 140
Number of sections extracted are: 79
Number of smaller articles are: 79


In [6]:
print(str(smaller_articles_count))

3746


In [7]:


print("Number of articles present are: " + str(total_drugs))
print("Number of sections extracted are: " + str(total_pulled))
print("Number of smaller articles are: " + str(smaller_articles_count))

Number of articles present are: 7346
Number of sections extracted are: 2844
Number of smaller articles are: 3746
